In [1]:
import json
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from typing import Dict, List, Tuple, Any
from collections import defaultdict

In [4]:
class SystemGraphBuilder:
    """
    Builds a directed graph from extracted system components and computes
    integration risk features.
    """
    
    def __init__(self):
        """Initialize the graph builder."""
        self.graph = None
        self.components = []
        self.features = {}
    
    def load_components(self, json_file: str) -> None:
        """
        Load extracted components from JSON file.
        
        Args:
            json_file: Path to extracted_components.json
        """
        print(f"📥 Loading components from {json_file}...")
        with open('extracted_components.json', 'r') as f:
            data = json.load(f)
        
        self.components = data['components']
        print(f"✅ Loaded {len(self.components)} components")
    
    def build_graph(self) -> nx.DiGraph:
        """
        Build a directed graph from components.
        
        Nodes represent components.
        Edges represent dependencies (A → B means A depends on B).
        
        Returns:
            NetworkX directed graph
        """
        print("🔨 Building dependency graph...")
        
        self.graph = nx.DiGraph()
        
        # Add nodes with attributes
        for comp in self.components:
            self.graph.add_node(
                comp['id'],
                name=comp['name'],
                type=comp['type'],
                technology=comp['technology'],
                purpose=comp['purpose'],
                criticality=comp['criticality']
            )
        
        # Add edges (dependencies)
        for comp in self.components:
            source = comp['id']
            for dependency in comp.get('dependencies', []):
                # Edge from source to dependency (source depends on dependency)
                if dependency in self.graph.nodes:
                    self.graph.add_edge(source, dependency)
                else:
                    print(f"⚠️  Warning: Dependency '{dependency}' not found for '{source}'")
        
        print(f"✅ Graph built: {self.graph.number_of_nodes()} nodes, {self.graph.number_of_edges()} edges")
        
        return self.graph
    
    def compute_features(self) -> Dict[str, Any]:
        """
        Compute graph-theoretic features for each node.
        
        Features computed:
        - In-degree: Number of components depending on this component
        - Out-degree: Number of components this component depends on
        - Total degree: In + Out degree
        - Betweenness centrality: How often node appears on shortest paths
        - Closeness centrality: How close node is to all other nodes
        - PageRank: Importance based on dependency structure
        - Clustering coefficient: Local connectivity
        
        Returns:
            Dictionary mapping component IDs to feature vectors
        """
        print("📊 Computing graph features...")
        
        if self.graph is None:
            raise ValueError("Graph not built. Call build_graph() first.")
        
        # Compute centrality measures
        betweenness = nx.betweenness_centrality(self.graph)
        closeness = nx.closeness_centrality(self.graph)
        pagerank = nx.pagerank(self.graph)
        clustering = nx.clustering(self.graph.to_undirected())
        
        # Build feature dictionary for each node
        self.features = {}
        for node in self.graph.nodes():
            self.features[node] = {
                'in_degree': self.graph.in_degree(node),
                'out_degree': self.graph.out_degree(node),
                'total_degree': self.graph.degree(node),
                'betweenness_centrality': betweenness[node],
                'closeness_centrality': closeness[node],
                'pagerank': pagerank[node],
                'clustering_coefficient': clustering[node],
                'criticality': self.graph.nodes[node]['criticality'],
                'type': self.graph.nodes[node]['type']
            }
        
        print("✅ Features computed for all nodes")
        
        return self.features
    
    def identify_critical_nodes(self, top_n: int = 5) -> List[Tuple[str, Dict]]:
        """
        Identify most critical nodes based on multiple metrics.
        
        Args:
            top_n: Number of top critical nodes to return
            
        Returns:
            List of (node_id, features) tuples sorted by criticality score
        """
        print(f"🎯 Identifying top {top_n} critical nodes...")
        
        if not self.features:
            raise ValueError("Features not computed. Call compute_features() first.")
        
        # Compute composite criticality score
        # Weighted combination of multiple features
        criticality_scores = {}
        
        for node, features in self.features.items():
            # Normalize and weight features
            score = (
                0.3 * features['in_degree'] +           # High in-degree = many depend on it
                0.2 * features['betweenness_centrality'] +  # High betweenness = bottleneck
                0.2 * features['pagerank'] +             # High pagerank = structurally important
                0.15 * features['out_degree'] +          # High out-degree = depends on many
                0.15 * (1 - features['clustering_coefficient'])  # Low clustering = less redundancy
            )
            criticality_scores[node] = score
        
        # Sort by score
        sorted_nodes = sorted(
            criticality_scores.items(),
            key=lambda x: x[1],
            reverse=True
        )
        
        top_nodes = [
            (node, self.features[node]) 
            for node, score in sorted_nodes[:top_n]
        ]
        
        print(f"✅ Identified {len(top_nodes)} critical nodes")
        
        return top_nodes
    
    def analyze_failure_impact(self, node: str) -> Dict[str, Any]:
        """
        Analyze the impact of a single node failure.
        
        Args:
            node: Component ID to analyze
            
        Returns:
            Dictionary with failure impact metrics
        """
        if node not in self.graph.nodes:
            raise ValueError(f"Node '{node}' not found in graph")
        
        # Components that directly depend on this node
        dependents = list(self.graph.predecessors(node))
        
        # Components this node depends on (would also fail)
        dependencies = list(self.graph.successors(node))
        
        # Cascade analysis: what fails if this node fails
        cascade = self._compute_failure_cascade(node)
        
        return {
            'node': node,
            'name': self.graph.nodes[node]['name'],
            'direct_dependents': len(dependents),
            'dependent_nodes': dependents,
            'dependencies': dependencies,
            'cascade_size': len(cascade),
            'cascade_nodes': cascade,
            'cascade_percentage': len(cascade) / self.graph.number_of_nodes() * 100
        }
    
    def _compute_failure_cascade(self, failed_node: str) -> List[str]:
        """
        Compute which nodes would fail if given node fails.
        Uses BFS to find all reachable predecessors.
        """
        cascade = set()
        queue = [failed_node]
        visited = {failed_node}
        
        while queue:
            current = queue.pop(0)
            # All nodes that depend on current node
            for predecessor in self.graph.predecessors(current):
                if predecessor not in visited:
                    visited.add(predecessor)
                    cascade.add(predecessor)
                    queue.append(predecessor)
        
        return list(cascade)
    
    def visualize_graph(self, output_file: str = "system_graph.png",
                       highlight_nodes: List[str] = None) -> None:
        """
        Visualize the dependency graph.
        
        Args:
            output_file: Output filename for the graph visualization
            highlight_nodes: List of node IDs to highlight in red
        """
        print(f"🎨 Visualizing graph...")
        
        if self.graph is None:
            raise ValueError("Graph not built. Call build_graph() first.")
        
        plt.figure(figsize=(16, 12))
        
        # Use spring layout for better visualization
        pos = nx.spring_layout(self.graph, k=2, iterations=50, seed=42)
        
        # Color nodes by type
        type_colors = {
            'application': '#3B82F6',  # Blue
            'infrastructure': '#10B981',  # Green
            'storage': '#F59E0B',  # Orange
            'network': '#8B5CF6',  # Purple
            'security': '#EF4444'  # Red
        }
        
        node_colors = []
        for node in self.graph.nodes():
            if highlight_nodes and node in highlight_nodes:
                node_colors.append('#DC2626')  # Bright red for highlighted
            else:
                node_type = self.graph.nodes[node]['type']
                node_colors.append(type_colors.get(node_type, '#6B7280'))
        
        # Size nodes by in-degree (how many depend on them)
        node_sizes = [
            300 + self.graph.in_degree(node) * 150 
            for node in self.graph.nodes()
        ]
        
        # Draw graph
        nx.draw_networkx_nodes(
            self.graph, pos,
            node_color=node_colors,
            node_size=node_sizes,
            alpha=0.9,
            edgecolors='white',
            linewidths=2
        )
        
        nx.draw_networkx_edges(
            self.graph, pos,
            edge_color='#9CA3AF',
            arrows=True,
            arrowsize=15,
            arrowstyle='->',
            width=1.5,
            alpha=0.6,
            connectionstyle='arc3,rad=0.1'
        )
        
        # Labels
        labels = {
            node: self.graph.nodes[node]['name'].replace(' ', '\n') 
            for node in self.graph.nodes()
        }
        nx.draw_networkx_labels(
            self.graph, pos,
            labels,
            font_size=7,
            font_weight='bold'
        )
        
        plt.title("LMS System Dependency Graph\n(Node size = # of dependents)", 
                 fontsize=16, fontweight='bold', pad=20)
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(output_file, dpi=300, bbox_inches='tight', facecolor='white')
        print(f"✅ Graph saved to {output_file}")
        plt.close()
    
    def save_features(self, output_file: str = "graph_features.json") -> None:
        """
        Save computed features to JSON file.
        
        Args:
            output_file: Output filename
        """
        if not self.features:
            raise ValueError("Features not computed. Call compute_features() first.")
        
        output = {
            'timestamp': '2025-10-30',
            'graph_statistics': {
                'num_nodes': self.graph.number_of_nodes(),
                'num_edges': self.graph.number_of_edges(),
                'density': nx.density(self.graph),
                'is_dag': nx.is_directed_acyclic_graph(self.graph)
            },
            'node_features': self.features
        }
        
        with open(output_file, 'w') as f:
            json.dump(output, f, indent=2)
        
        print(f"💾 Features saved to {output_file}")



In [5]:
def main():
    """
    Main execution - demonstrates the graph building pipeline.
    
    NOTE: Before running this, ensure you have:
    1. Created extracted_components.json using ChatGPT prompts
    2. Validated the JSON structure
    3. Verified all dependencies reference existing components
    """
    print("="*60)
    print("LMS SYSTEM GRAPH BUILDER")
    print("="*60)
    print("\nPrerequisites:")
    print("✓ extracted_components.json (from ChatGPT/Claude)")
    print("✓ JSON validated and verified")
    print()
    
    # Initialize builder
    builder = SystemGraphBuilder()
    
    # Load components from extractor output (manually created using LLM)
    builder.load_components('extracted_components.json')
    
    # Build graph
    builder.build_graph()
    
    # Compute features
    builder.compute_features()
    
    # Identify critical nodes
    critical_nodes = builder.identify_critical_nodes(top_n=5)
    
    print("\n" + "="*60)
    print("TOP 5 CRITICAL COMPONENTS")
    print("="*60)
    for i, (node, features) in enumerate(critical_nodes, 1):
        print(f"{i}. {builder.graph.nodes[node]['name']}")
        print(f"   In-degree: {features['in_degree']} (components depending on this)")
        print(f"   Out-degree: {features['out_degree']} (dependencies)")
        print(f"   Betweenness: {features['betweenness_centrality']:.3f}")
        print(f"   PageRank: {features['pagerank']:.3f}")
        print()
    
    # Analyze failure impact for top critical node
    top_node = critical_nodes[0][0]
    impact = builder.analyze_failure_impact(top_node)
    
    print("="*60)
    print(f"FAILURE IMPACT ANALYSIS: {impact['name']}")
    print("="*60)
    print(f"Direct dependents: {impact['direct_dependents']}")
    print(f"Cascade size: {impact['cascade_size']} components")
    print(f"Cascade percentage: {impact['cascade_percentage']:.1f}% of system")
    print()
    
    # Save results
    builder.save_features()
    
    # Visualize graph (highlight critical nodes)
    critical_node_ids = [node for node, _ in critical_nodes]
    builder.visualize_graph(highlight_nodes=critical_node_ids)
    
    print("="*60)
    print("✅ Graph building and feature computation complete!")
    print("="*60)


if __name__ == "__main__":
    main()

LMS SYSTEM GRAPH BUILDER

Prerequisites:
✓ extracted_components.json (from ChatGPT/Claude)
✓ JSON validated and verified

📥 Loading components from extracted_components.json...
✅ Loaded 22 components
🔨 Building dependency graph...
✅ Graph built: 22 nodes, 38 edges
📊 Computing graph features...
✅ Features computed for all nodes
🎯 Identifying top 5 critical nodes...
✅ Identified 5 critical nodes

TOP 5 CRITICAL COMPONENTS
1. Primary Database Cluster
   In-degree: 10 (components depending on this)
   Out-degree: 0 (dependencies)
   Betweenness: 0.000
   PageRank: 0.180

2. Cache Layer
   In-degree: 6 (components depending on this)
   Out-degree: 0 (dependencies)
   Betweenness: 0.000
   PageRank: 0.111

3. Object Storage
   In-degree: 5 (components depending on this)
   Out-degree: 0 (dependencies)
   Betweenness: 0.000
   PageRank: 0.092

4. API Gateway Service
   In-degree: 1 (components depending on this)
   Out-degree: 7 (dependencies)
   Betweenness: 0.024
   PageRank: 0.039

5. Appl